In [1]:
#-*- coding: utf-8 -*-
%matplotlib inline
from matplotlib.font_manager import FontProperties  
import matplotlib.pyplot as plt  
import numpy as np
import pandas as pd
import os

plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号


/Users/ppj/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
pieces=[]
columns=['ID','warning','empty_taxi','dome_light'
                        ,'elevated','brake','received_time','GPS_time','Longitude','Latitude','speed','direction','satellite']
for sample in range(4):
    path='/Users/ppj/Desktop/taxi_data/part-0000%d.csv' %sample
    frame=pd.read_csv(path,names=columns)
    pieces.append(frame)
taxi=pd.concat(pieces,ignore_index=True)

In [3]:
taxi.GPS_time=taxi.GPS_time.map(lambda x: x[11:])

In [4]:
taxi_mor=taxi[(taxi.GPS_time>='07:30:00')&(taxi.GPS_time<='09:30:00')]
taxi_even=taxi[(taxi.GPS_time>='16:30:00')&(taxi.GPS_time<='19:00:00')]
taxi_midnight=taxi[(taxi.GPS_time>='00:00:00')&(taxi.GPS_time<='04:00:00')]
taxi_other=taxi[((taxi.GPS_time>='04:00:00')&(taxi.GPS_time<='07:30:00'))+((taxi.GPS_time>='09:30:00')&(taxi.GPS_time<='12:30:00'))+
               ((taxi.GPS_time>='19:00:00')&(taxi.GPS_time<='23:59:59'))]

/Users/ppj/anaconda/lib/python2.7/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


In [5]:
taxi_se=taxi.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_even_se=taxi_even.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_mor_se=taxi_mor.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_midnight_se=taxi_midnight.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_other_se=taxi_other.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]

In [7]:
ID_unique=taxi_se.ID.unique()

In [8]:
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
from itertools import cycle

In [9]:
from math import radians, cos, sin, asin, sqrt  
  
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000  


In [10]:
def plot_cluster(location,taxi_id):
	# The following bandwidth can be automatically detected using
	#bandwidth = estimate_bandwidth(location, quantile=0.1)
    bandwidth = 0.09
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, min_bin_freq=5)
    ms.fit(location)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_        #每一类聚合的中心点
    
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    max_len=len(labels[labels==0])
    k=0
    
    for i in range(n_clusters_):
        s=len(labels[labels==i])
        if s>=max_len:
            max_len=s
            k=i
    max_center=cluster_centers[k]
    max_r=haversine(location[0][0],location[0][1],max_center[0],max_center[1])
    for j in range(len(location)):
        sa=haversine(location[j][0],location[j][1],max_center[0],max_center[1])
        if sa>=max_r:
            max_r=sa
    
        
    
    
    return [taxi_id,cluster_centers[k][0],cluster_centers[k][1],max_r]
    '''
    print("number of estimated clusters : %d" % n_clusters_)

    fig = plt.figure()
    ax=fig.add_subplot(1,1,1)
    colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
    for k, col in zip(range(n_clusters_), colors) :
        my_members=labels==k    #就是把同种标签的坐标都取出来，并将同种标签的都打上同种颜色
        cluster_center = cluster_centers[k]
        plt.plot(location[my_members, 0],location[my_members, 1], col + '.')
        plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,markeredgecolor='k',markersize=7)
    plt.title('Estimated number of clusters: %d,the bandwidth: %.4f' % (n_clusters_,bandwidth))
    plt.savefig('cluster_taxi_%s.jpg' %taxi_id)
    '''

In [11]:
def get_location(taxi_id):
    #taxi_location=np.array(taxi_se[taxi_se.ID==taxi_id].loc[:,['Longitude','Latitude']])
    taxi_mor_location=np.array(taxi_mor_se[taxi_mor_se.ID==taxi_id].loc[:,['Longitude','Latitude']])
    taxi_midnight_location=np.array(taxi_midnight_se[taxi_midnight_se.ID==taxi_id].loc[:,['Longitude','Latitude']])
    taxi_even_location=np.array(taxi_even_se[taxi_even_se.ID==taxi_id].loc[:,['Longitude','Latitude']])
    taxi_other_location=np.array(taxi_other_se[taxi_other_se.ID==taxi_id].loc[:,['Longitude','Latitude']])
    
    collect_location=[taxi_mor_location,taxi_midnight_location,taxi_even_location,taxi_other_location]
    collect_flag=['morning','midnight','evening','other_time']
    collect_centers=[]
    for i in range(len(collect_location)):
        if len(collect_location[i])>5:
            sma=plot_cluster(collect_location[i],taxi_id)
            sma.append(collect_flag[i])
            collect_centers.append(sma)
    return collect_centers
    
    
    

In [12]:
get_location(ID_unique[1])

[[27320, 121.39784593750001, 31.078757093749996, 6594.328105198569, 'morning'],
 [27320, 121.42272194736849, 31.007241513157876, 6.97274028946498, 'midnight'],
 [27320, 121.56731999999998, 31.279214515151512, 13745.25180827502, 'evening'],
 [27320,
  121.41298361788621,
  31.028894804878057,
  28355.57161256304,
  'other_time']]

In [34]:
get_cluster_center=[]
for ID in ID_unique[0:50]:
    dataframe_id=pd.DataFrame(get_location(ID))
    get_cluster_center.append(dataframe_id)
id_cluster_center=pd.concat(get_cluster_center,ignore_index=True)

In [35]:
id_cluster_center.columns=['ID','cluster_Longitude','cluster_Latitude','radius','time']

In [65]:
id_cluster_center.to_csv('id_cluster_center.csv')

In [37]:
id_cluster_center

,ID,cluster_Longitude,cluster_Latitude,radius,time
0,18834,121.590106,31.131552,6.446586,morning
1,18834,121.590110,31.131556,15.205451,midnight
2,18834,121.610383,31.248821,11095.569389,evening
3,18834,121.584178,31.134814,22257.400384,other_time
4,27320,121.397846,31.078757,6594.328105,morning
5,27320,121.422722,31.007242,6.972740,midnight
6,27320,121.567320,31.279215,13745.251808,evening
7,27320,121.412984,31.028895,28355.571613,other_time
8,16926,121.615634,31.289030,24964.763338,morning
9,16926,121.640098,31.288574,21665.876161,midnight


In [38]:
sss=id_cluster_center[id_cluster_center.time!='other_time'][id_cluster_center.ID==ID_unique[1]]

/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [39]:
sss.iloc[1,:]

ID                      27320
cluster_Longitude     121.423
cluster_Latitude      31.0072
radius                6.97274
time                 midnight
Name: 5, dtype: object

In [40]:
range(len(sss))

[0, 1, 2]

In [41]:
max_radius=sss.iloc[0,:].radius
m=0
for p in range(len(sss)):
    s=sss.iloc[p,:].radius
    if s>=max_radius:
        max_radius=s
        m=p
print sss.iloc[m,:].time

evening


In [42]:
def analysis_time_like(taxi_ID):
    id_time_like=id_cluster_center[id_cluster_center.time!='other_time'][id_cluster_center.ID==taxi_ID]
    max_radius=id_time_like.iloc[0,:].radius
    m=0
    for p in range(len(id_time_like)):
        s=id_time_like.iloc[p,:].radius
        if s>=max_radius:
            max_radius=s
            m=p
    return [taxi_ID,id_time_like.iloc[m,:].time]

In [55]:
collcet_time=[]
for ID_time in ID_unique[0:50]:
    collcet_time.append(analysis_time_like(ID_time))


/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [56]:
pd.DataFrame(collcet_time,columns=['ID','Like_time']).to_csv('ID_like_time.csv')

In [58]:
driver=pd.read_csv('driver.csv')
ID_like_time=pd.read_csv('ID_like_time.csv')

In [59]:
driver_information=driver.merge(ID_like_time)

In [60]:
del driver_information['Unnamed: 0']

In [61]:
driver_information.to_csv('driver_information.csv')

In [62]:
type(driver_information.sort_index(by='historical_std').ID[1])

/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


numpy.int64

In [63]:
sample_list=[13468.0,15990.0,20227.0,25439,18204.0,21615,25242.0,25413.0,13411.0,11808]

In [64]:
want_sample=driver_information.sort_index(by='historical_std')

/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  if __name__ == '__main__':


In [54]:
want_sample

,ID,Historical_rate,historical_std,max_steady,min_steady,Like_time
6,10827.0,5.414286,2.979779,other_time,evening,evening
19,13468.0,28.537500,13.407905,other_time,morning,evening
3,26337.0,30.156863,13.589749,other_time,evening,evening
13,15990.0,25.467500,13.793574,other_time,morning,morning
34,10558.0,26.697500,15.195386,evening,morning,morning
23,20227.0,34.122449,15.313903,morning,evening,morning
5,23390.0,31.351064,15.506757,other_time,morning,midnight
11,14891.0,27.490741,15.753345,morning,evening,morning
29,11808.0,23.925000,15.799993,other_time,evening,midnight
16,21318.0,29.960784,15.889164,other_time,evening,morning


In [52]:
np.array(want_sample.set_index('ID').ix[sample_list].reset_index())

array([[13468.0, 28.5375, 13.4079053951, 'other_time', 'morning', 'evening'],
       [15990.0, 25.4675, 13.793574364500001, 'other_time', 'morning',
        'morning'],
       [20227.0, 34.1224489796, 15.3139027845, 'morning', 'evening',
        'morning'],
       [25439.0, 44.0661016949, 26.3210144192, 'other_time', 'morning',
        'evening'],
       [18204.0, 41.9780487805, 20.7701651906, 'other_time', 'evening',
        'evening'],
       [21615.0, 37.0612903226, 20.4619659356, 'evening', 'evening',
        'morning'],
       [25242.0, 40.138461538499996, 22.965168892199998, 'morning',
        'evening', 'morning'],
       [25413.0, 38.8275862069, 19.8466517032, 'other_time', 'evening',
        'evening'],
       [13411.0, 38.0185185185, 24.5601987119, 'other_time', 'midnight',
        'evening'],
       [11808.0, 23.925, 15.7999934072, 'other_time', 'evening', 'midnight']], dtype=object)

In [53]:
np.array(id_cluster_center.set_index('ID').ix[sample_list].reset_index())

array([[13468, 121.49412693939395, 31.301707515151513, 6180.264251754371,
        'morning'],
       [13468, 121.48843652564105, 31.34981208974361, 11.404035777922227,
        'midnight'],
       [13468, 121.49426492452832, 31.286262584905664, 14213.373017598618,
        'evening'],
       [13468, 121.47635346323528, 31.344758455882364, 21891.474072059904,
        'other_time'],
       [15990, 121.38505582051283, 31.299647948717947, 8746.286328431217,
        'morning'],
       [15990, 121.39390689743585, 31.3046343974359, 2329.1927671082317,
        'midnight'],
       [15990, 121.41462907142854, 31.303121119047617, 4458.0603075887375,
        'evening'],
       [15990, 121.4025402994923, 31.295025868020304, 14549.619749203202,
        'other_time'],
       [20227, 121.48596596666663, 31.218502833333332, 7036.828673957698,
        'morning'],
       [20227, 121.42395141428577, 31.349397442857143, 3.707320463119825,
        'midnight'],
       [20227, 121.39827574468083, 31.19383608510